In [29]:
from paho.mqtt import client as mqtt_client
from datetime import datetime
import csv
import os
import json
from adtk.detector import SeasonalAD
from adtk.data import validate_series
import pandas as pd
from adtk.detector import LevelShiftAD
from adtk.detector import QuantileAD
from sklearn.neighbors import LocalOutlierFactor
from adtk.detector import OutlierDetector
import warnings
warnings.filterwarnings("ignore", message="Setting an item of incompatible dtype")

In [20]:
def sanitize_topic_name(topic):
    # Replace invalid characters with underscores
    sanitized_topic = topic.replace('/', '_')
    return sanitized_topic

def on_message_data_append(topic, row):
    # Sanitize the topic name to ensure it can be used in file paths
    csv_file = f"data_csv_{topic}.csv"

    if not os.path.exists(csv_file):
        # If the file does not exist, create it and write the header
        with open(csv_file, mode='w', newline='') as file:
            file.write('date,x,y,z\n')
    
    # Read the CSV file into a DataFrame or create an empty DataFrame if the file is just created
    df = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['date', 'x', 'y', 'z'])

    size = len(df)

    # Check the row count and drop the oldest row if necessary
    if len(df) >= 120:
        df = df.drop(0)

    # Create a DataFrame with the new row
    new_df = pd.DataFrame([{'date': row[0], 'x': row[1], 'y': row[2], 'z': row[3]}])

    # Concatenate the existing DataFrame with the new DataFrame
    new_df = pd.concat([df, new_df], ignore_index=True)

    # Save the new DataFrame back to the CSV file
    new_df.to_csv(csv_file, index=False)

    return size


In [21]:
def getanomalyList(anomalies):
    
    # Filter out NaN values from anomalies
    non_nan_anomalies = anomalies.dropna()
    
    anomalies_dates = []
    
    # Append the dates of the anomalies of the x-axis
    for anomaly_idx, anomaly in non_nan_anomalies.items():
        if anomaly:
            anomalies_dates.append(anomaly_idx)

    return anomalies_dates


In [22]:
def getAnomalysLevelShift(topic):
    
    csv_file = f"data_csv_{topic}.csv"
    
    sleeping_data = pd.read_csv(csv_file)

    # Convert 'date' column to datetime
    sleeping_data['date'] = pd.to_datetime(sleeping_data['date'])

    # Set 'date' column as the index
    sleeping_data.set_index('date', inplace=True)

    # get each axis data
    x_axis_data = sleeping_data["x"]
    y_axis_data = sleeping_data["y"]
    z_axis_data = sleeping_data["z"]
    
    # validating series
    x_train = validate_series(x_axis_data)
    y_train = validate_series(y_axis_data)
    z_train = validate_series(z_axis_data)

    # Create and fit the LevelShiftAD detector for x_train
    level_shift_ad_x = LevelShiftAD(c=6.0, side='both', window=3)
    anomalies_x = level_shift_ad_x.fit_detect(x_train)
    anomalies_x_dates = getanomalyList(anomalies_x) # list of anomalies
    
    # Create and fit the LevelShiftAD detector for y_train
    level_shift_ad_y = LevelShiftAD(c=6.0, side='both', window=3)
    anomalies_y = level_shift_ad_y.fit_detect(y_train)
    anomalies_y_dates = getanomalyList(anomalies_y) # list of anomalies
    
    # Create and fit the LevelShiftAD detector for z_train
    level_shift_ad_z = LevelShiftAD(c=6.0, side='both', window=3)
    anomalies_z = level_shift_ad_z.fit_detect(z_train)
    anomalies_z_dates = getanomalyList(anomalies_z) # list of anomalies
    
    return anomalies_x_dates, anomalies_y_dates, anomalies_z_dates
    


In [23]:
def getAnomalyQuantile(topic):
    
    csv_file = f"data_csv_{topic}.csv"
    
    sleeping_data = pd.read_csv(csv_file)

    # Convert 'date' column to datetime
    sleeping_data['date'] = pd.to_datetime(sleeping_data['date'])

    # Set 'date' column as the index
    sleeping_data.set_index('date', inplace=True)

    # get each axis data
    x_axis_data = sleeping_data["x"]
    y_axis_data = sleeping_data["y"]
    z_axis_data = sleeping_data["z"]
    
    # validating series
    x_train = validate_series(x_axis_data)
    y_train = validate_series(y_axis_data)
    z_train = validate_series(z_axis_data)

    # Create and fit the QuantileAD detector for x_train
    quantile_ad_x = QuantileAD(high=0.99, low=0.02)
    anomalies_x = quantile_ad_x.fit_detect(x_train)
    anomalies_x_dates = getanomalyList(anomalies_x) # list of anomalies

    # Create and fit the QuantileAD detector for y_train
    quantile_ad_y = QuantileAD(high=0.99, low=0.02)
    anomalies_y = quantile_ad_y.fit_detect(y_train)
    anomalies_y_dates = getanomalyList(anomalies_y) # list of anomalies


    # Create and fit the QuantileAD detector for z_train
    quantile_ad_z = QuantileAD(high=0.99, low=0.02)
    anomalies_z = quantile_ad_z.fit_detect(z_train)
    anomalies_z_dates = getanomalyList(anomalies_z) # list of anomalies

    return anomalies_x_dates, anomalies_y_dates, anomalies_z_dates
    

In [24]:
def getOutlierDetection(topic):
    
    csv_file = f"data_csv_{topic}.csv"
    
    sleeping_data = pd.read_csv(csv_file)

    # Convert 'date' column to datetime
    sleeping_data['date'] = pd.to_datetime(sleeping_data['date'])

    # Set 'date' column as the index
    sleeping_data.set_index('date', inplace=True)

    # get each axis data
    x_axis_data = sleeping_data["x"]
    y_axis_data = sleeping_data["y"]
    z_axis_data = sleeping_data["z"]
    
    # validating series
    x_train = validate_series(x_axis_data)
    y_train = validate_series(y_axis_data)
    z_train = validate_series(z_axis_data)

    x_train_df = pd.DataFrame(x_train)
    y_train_df = pd.DataFrame(y_train)
    z_train_df = pd.DataFrame(z_train)
    
    outlier_detector_x = OutlierDetector(LocalOutlierFactor(contamination=0.02))
    anomalies_x = outlier_detector_x.fit_detect(x_train_df)
    anomalies_x_dates = getanomalyList(anomalies_x) # list of anomalies
    
    outlier_detector_y = OutlierDetector(LocalOutlierFactor(contamination=0.02))
    anomalies_y = outlier_detector_y.fit_detect(y_train_df)
    anomalies_y_dates = getanomalyList(anomalies_y) # list of anomalies
   
    
    outlier_detector_z = OutlierDetector(LocalOutlierFactor(contamination=0.02))
    anomalies_z = outlier_detector_z.fit_detect(z_train_df)
    anomalies_z_dates = getanomalyList(anomalies_z) # list of anomalies

    return anomalies_x_dates, anomalies_y_dates, anomalies_z_dates
    

In [25]:
def check_anomaly_in_rows(anomaly_timestamps,topic, rows):

    # Check if the list is empty
    if not anomaly_timestamps:
        return False
        
    df = pd.read_csv(f"data_csv_{topic}.csv")

    df['date'] = pd.to_datetime(df['date'])

    # Set 'date' column as the index
    df.set_index('date', inplace=True)

    # Get the last 6 rows of the data
    last_rows = df.tail(rows)

    # Check if any of the anomaly timestamps exist within the last 6 seconds of the record
    for anomaly_timestamp in anomaly_timestamps:
        if anomaly_timestamp in last_rows.index:
            return True
            
    return False

In [26]:
def at_least_two_true(b1, b2, b3):
  # Count the number of True values
  true_count = sum([b1, b2, b3])
  return true_count >= 2

In [27]:
def getDetection(topic,row):
    
        anomalies_x_dates_ls, anomalies_y_dates_ls, anomalies_z_dates_ls = getAnomalysLevelShift(topic)
        anomalies_x_dates_q, anomalies_y_dates_q, anomalies_z_dates_q = getAnomalyQuantile(topic)
        anomalies_x_dates_od, anomalies_y_dates_od, anomalies_z_dates_od = getOutlierDetection(topic)

        check_x_ls = check_anomaly_in_rows(anomalies_x_dates_ls,topic,6)
        check_y_ls = check_anomaly_in_rows(anomalies_y_dates_ls,topic,6)
        check_z_ls = check_anomaly_in_rows(anomalies_z_dates_ls,topic,6)
        LevelShiftAD = at_least_two_true(check_x_ls, check_y_ls, check_z_ls)

        check_x_q = check_anomaly_in_rows(anomalies_x_dates_q,topic,6)
        check_y_q = check_anomaly_in_rows(anomalies_y_dates_q,topic,6)
        check_z_q = check_anomaly_in_rows(anomalies_z_dates_q,topic,6)
        QuantileAD = at_least_two_true(check_x_q, check_y_q, check_z_q)
     
        check_x_od = check_anomaly_in_rows(anomalies_x_dates_od,topic,6)
        check_y_od = check_anomaly_in_rows(anomalies_y_dates_od,topic,6)
        check_z_od = check_anomaly_in_rows(anomalies_z_dates_od,topic,6)
        OutlierDetection = at_least_two_true(check_x_od, check_y_od, check_z_od)

        return at_least_two_true(LevelShiftAD,QuantileAD,OutlierDetection)

In [ ]:
broker = 'broker.emqx.io'
port = 1883
topic = "move_id/AA"
client_id = ''
topic_clear = sanitize_topic_name(topic)

def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def subscribe(client: mqtt_client):
    #os.remove(f"data_csv_{topic_clear}.csv")
    def on_message(client, userdata, msg): 
        
        time = datetime.now()
        data = json.loads(msg.payload.decode())
        x = float(data['accelerometerSensor']['x'])
        y = float(data['accelerometerSensor']['y'])
        z = float(data['accelerometerSensor']['z'])
        row = [time, x, y, z]

        lendef = on_message_data_append(topic_clear,row)
        print(lendef)
        
        if(lendef > 119):
            if (getDetection(topic_clear,row)):
                print("ALERTA O GAJO LEVANTOU - SE !!!!!")
            else:
                print("o gajo esta fixe")
        
        
    client.subscribe(topic)
    client.on_message = on_message


def run():
    client = connect_mqtt()
    subscribe(client)
    client.loop_forever()


if __name__ == '__main__':
    run()

C:\Users\hugod\AppData\Local\Temp\ipykernel_17804\4200100245.py:14: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt_client.Client(mqtt_client.CallbackAPIVersion.VERSION1,client_id)


Connected to MQTT Broker!
120
o gajo esta fixe
120
o gajo esta fixe
120
o gajo esta fixe
120
o gajo esta fixe
120
o gajo esta fixe
120
o gajo esta fixe
120
o gajo esta fixe
120
o gajo esta fixe
120
ALERTA O GAJO LEVANTOU - SE !!!!!
